# Convert FSC MINT Metadata to YAML

In [1]:
import mint_client
import requests
import json
from ast import literal_eval
import yaml

In [2]:
configuration = mint_client.Configuration()
api_instance = mint_client.UserApi()

username = 'modelservice' # str | The user name for login
password = 'development' # str | The password for login in clear text
user = mint_client.User(username=username, password=password) # User | Created user object
configuration.access_token = api_instance.login_user(username, password)

### Description

In [3]:
api_instance = mint_client.ModelApi(mint_client.ApiClient(configuration))
api_response = api_instance.get_models(username=username)

In [4]:
description = [i for i in api_response if i.id=='FSC'][0]

In [5]:
description

{'description': 'The Food Shocks Cascade Model (FSC) is a simple agent-based '
                'network model that computes chain-reactions due to negative '
                'production anomalies based on dynamic food balance sheets at '
                'the country level.',
 'has_documentation': ['https://github.com/mjpuma/FSC-WorldModelers'],
 'has_model_category': ['Economic'],
 'has_software_version': [{'id': 'FSC_0.1',
                           'type': ['http://ontosoft.org/software#SoftwareVersion',
                                    'https://w3id.org/mint/modelCatalog#ModelVersion']}],
 'id': 'FSC',
 'label': 'FSC',
 'type': ['https://w3id.org/mint/modelCatalog#TheoryBasedModel',
          'https://w3id.org/mint/modelCatalog#Model']}

### Versions/Configs

In [6]:
api_instance = mint_client.ModelversionApi(mint_client.ApiClient(configuration))
version = api_instance.get_model_version('FSC', username=username)

In [7]:
version

{'id': 'FSC_0.1',
 'type': ['http://ontosoft.org/software#SoftwareVersion',
          'https://w3id.org/mint/modelCatalog#ModelVersion']}

In [8]:
api_instance = mint_client.ModelconfigurationApi(mint_client.ApiClient(configuration))
config_meta = api_instance.get_model_configuration(version.id, username=username)

In [9]:
config_meta

{'id': 'FSC_config',
 'type': ['mc:ModelConfiguration',
          'https://w3id.org/mint/modelCatalog#ModelConfiguration']}

#### Get configuration

In [10]:
api_instance = mint_client.ModelconfigurationApi(mint_client.ApiClient(configuration))
config = api_instance.get_model_configuration(config_meta.id, username=username)

### Get inputs and outputs

In [11]:
api_instance = mint_client.DatasetspecificationApi(mint_client.ApiClient(configuration))

In [12]:
datasets = api_instance.get_data_sets(username=username)

In [13]:
input_ids = [i.id for i in config.has_input]
output_ids = [i.id for i in config.has_output]

In [14]:
inputs = [i for i in datasets if i.id in input_ids]
outputs = [i for i in datasets if i.id in output_ids]

### Get variables

> Note: this relies on MaaS for now for expediency

In [15]:
url = "https://model-service.worldmodelers.com/model_io"
headers = {"accept": "application/json",
           "Content-Type": "application/json"}
payload = {"iotype":"input",
           "name":"FSC"}

In [17]:
resp = requests.post(url, headers=headers, json=payload, auth=('wmuser','Bq248mQHV9EDuUap'))

inputs = resp.json()

In [18]:
payload['iotype'] = 'output'
resp = requests.post(url, headers=headers, json=payload, auth=('wmuser','Bq248mQHV9EDuUap'))
outputs = resp.json()

In [19]:
def find_ontology(uri):
    if 'agrovoc' in uri:
        return 'AGROVOC'
    elif 'geoscienceontology' in uri:
        return 'SVO'
    elif 'schema.org' in uri:
        return 'schema.org'
    else:
        return ''
    
def get_variables(file):
    out_vars = []
    in_vars = file['variables']
    for vv in in_vars:
        vv_out = {'name': vv['name'],
                  'description': vv.get('metadata',{}).get('description',''),
                  'standard_variables': []
                 }
        
        for sv in vv['standard_names']:
            sv.pop('standard_variable_id')
            sv['name'] = sv.pop('standard_variable_name')
            sv['uri'] = sv.pop('standard_variable_uri')
            sv['ontology'] = find_ontology(sv['uri'])
            vv_out['standard_variables'].append(sv)
        
        out_vars.append(vv_out)
    return out_vars

In [20]:
def format_io_file(file):
    f = {}
    f['name'] = file['name']
    f['format'] = file['filetype']
    f['description'] = file['description']
    f['variables'] = get_variables(file)
    return f

### Obtain Parameters

In [21]:
api_instance = mint_client.ParameterApi(mint_client.ApiClient(configuration))

In [22]:
parameters = api_response = api_instance.get_parameters(username=username)

In [23]:
pars = [i for i in parameters if 'FSC' in i.id]

In [24]:
def get_parameter_type(par):
    if 'string' in par.has_data_type:
        return "StringParameter"
    elif 'number' in par.has_data_type:
        return "NumberParameter"
    else:
        return ""

In [25]:
pars_ = []
for par in pars:
    par_ = {}
    par_['name'] = par.label
    par_['description'] = par.description
    par_['metadata'] = {'type': get_parameter_type(par)}
    par_['default'] = literal_eval(par.has_default_value)[0]
    
    pars_.append(par_)

In [26]:
pars_

[{'name': 'country',
  'description': 'The ISO 3 country code for the country of interest.',
  'metadata': {'type': 'StringParameter'},
  'default': 'SOM'},
 {'name': 'fractional_reserve_access',
  'description': 'The percentage of fractional reserves which may be accessed (from 0 to 1, where 1 equals a 100% decrease)',
  'metadata': {'type': 'NumberParameter'},
  'default': '0.3'},
 {'name': 'production_decrease',
  'description': 'The decrease in production you wish to induce (from 0 to 1, where 1 equals a 100% decrease)',
  'metadata': {'type': 'NumberParameter'},
  'default': '0.2'},
 {'name': 'year',
  'description': 'The year you wish to simulate',
  'metadata': {'type': ''},
  'default': '2005'}]

### Generate Output Model

In [27]:
model = {}

In [35]:
model['id'] = description.id
model['name'] = description.id
model['maintainer'] = {'name': 'Michael Puma', 'email': 'mjp38@columbia.edu'}
model['author'] = [{'name': 'Michael Puma', 'email': 'mjp38@columbia.edu'}]
model['version'] = description.has_software_version[0]['id']

model['description'] = description.description
model['website'] = description.has_documentation[0]

In [29]:
model['inputs'] = [format_io_file(i) for i in inputs]
model['outputs'] = [format_io_file(i) for i in outputs]

In [30]:
model['parameters'] = pars_

### Dump to YAML

In [36]:
class MyDumper(yaml.Dumper):

    def increase_indent(self, flow=False, indentless=False):
        return super(MyDumper, self).increase_indent(flow, False)

In [38]:
ff = open('FSC-metadata.yaml', 'w+')
yaml.dump(model, ff, Dumper=MyDumper, sort_keys=False, indent=2)